In [28]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition
import psycopg2

## Conectar Base de datos y Preparacion de los datos a utilizar

In [29]:
conexion = psycopg2.connect(
        dbname="buap",
        user="postgres",
        password="contrasena",
        host="localhost",
        port="5432"
    )
cursor = conexion.cursor()
datos = {}

In [30]:
# Profesores (con nombre)
cursor.execute("SELECT id_profesor, nombre, id_contrato FROM Profesor limit 100 ")
profesores = cursor.fetchall()
datos['Id_Profesores'] = [row[0] for row in profesores] # Lista de IDs de profesores
datos['nombres_profesor'] = {row[0]: row[1] for row in profesores} # Diccionario de nombres de profesores
datos['Tipo_Contrato'] = {row[0]: row[2] for row in profesores} # Diccionario de tipo de contrato de cada profesor

In [31]:
#TIpos de contratos 
cursor.execute("SELECT id_contrato, horas_minimas, horas_maximas FROM Contrato WHERE id_contrato = ANY(%s)", (list(datos['Tipo_Contrato'].values()),))
contratos_detalles = cursor.fetchall()
datos['Detalles_Contratos'] = {row[0]: {'min': row[1], 'max': row[2]} for row in contratos_detalles}  # Detalles por id_contrato
datos['Detalles_Contratos'][4] = {'min': 0, 'max': 99}  # Contrato de profesor imaginario

In [32]:
# Materias (con nombre)
cursor.execute("SELECT id_materia, nombre, horas_por_semana, id_tipo_clase, id_programa_educativo FROM Materia limit 100")
materias = cursor.fetchall()
datos['id_Materias'] = [row[0] for row in materias] # Lista de IDs de materias
datos['nombres_materia'] = {row[0]: row[1] for row in materias} # Diccionario de nombres de materias
datos['Horas_Semana_Materia'] = {row[0]: row[2] for row in materias} # Diccionario de horas por semana de cada materia
datos['Tipo_Clase'] = {row[0]: row[3] for row in materias} # Diccionario de tipo de clase de cada materia
datos['Programa_Educativo_Materia'] = {row[0]: row[4] for row in materias} # Diccionario de programa educativo de cada materia

In [33]:
# Aulas
cursor.execute("SELECT id_aula, id_edificio, id_tipo_clase FROM Aula ")
rows = cursor.fetchall()

datos['Aulas'] = [(row[0], row[1]) for row in rows] # Lista de IDs compuestos (id_aula, id_edificio)
datos['Tipo_Aula'] = {(row[0], row[1]): row[2] for row in rows}# Diccionario: clave compuesta (id_aula, id_edificio) -> tipo clase

# Edificios 
cursor.execute("SELECT id_edificio, nombre FROM Edificio")
rows = cursor.fetchall()

datos['Id_Edificios'] = [row[0] for row in rows]          # Lista con todos los id_edificio
datos['Edificios'] = {row[0]: row[1] for row in rows}     # Diccionario id_edificio -> nombre



# Bloques horarios
datos['Dia'] = list(range(1, 6)) # Lunes a Viernes
datos['Hora'] = list(range(1, 11)) # 10 bloques horarios por día/ de 8 am a 6 pm

In [34]:
aulas_param = datos['Aulas']  # lista de tuplas (id_aula, id_edificio)

query = """
SELECT A.id_aula, A.id_edificio, M.id_materia,
       CASE WHEN A.id_tipo_clase = M.id_tipo_clase THEN 1 ELSE 0 END AS compatible,
       M.id_tipo_clase AS tipo_materia,
       A.id_tipo_clase AS tipo_aula
FROM Aula A
CROSS JOIN Materia M
WHERE (A.id_aula, A.id_edificio) = ANY(%s) AND M.id_materia = ANY(%s)
"""

cursor.execute(query, (aulas_param, datos['id_Materias']))

datos['Compatibiliadad_Aula_Materia'] = {
    ((row[0], row[1]), row[2]): {
        'compatible': row[3],
        'tipo_materia': row[4],
        'tipo_aula': row[5]
    }
    for row in cursor.fetchall()
}


In [35]:
from collections import defaultdict
import unicodedata
import re

def normalizar_nombre(nombre):
    nombre = nombre.lower()
    nombre = ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    )
    nombre = nombre.strip()
    nombre = re.sub(r'\s+', ' ', nombre)
    return nombre

datos['Materias_por_Profesor'] = defaultdict(list)
datos['Profesores_por_Materia'] = defaultdict(list)
datos['Profesores_Imaginarios'] = []
datos['Profesores_Totales'] = []

# Crear dict normalizado nombre -> lista ids materias
materias_por_nombre = defaultdict(list)
for mid in datos['id_Materias']:
    nombre_original = datos['nombres_materia'][mid]
    nombre_norm = normalizar_nombre(nombre_original)
    materias_por_nombre[nombre_norm].append(mid)

cursor.execute("""
    SELECT id_profesor, id_materia 
    FROM profesor_materia
    WHERE id_profesor = ANY(%s) AND id_materia = ANY(%s)
""", (datos['Id_Profesores'], datos['id_Materias']))

for profe_id, materia_id in cursor.fetchall():
    nombre_original = datos['nombres_materia'][materia_id]
    nombre_norm = normalizar_nombre(nombre_original)
    materias_similares = materias_por_nombre[nombre_norm]
    for mid in materias_similares:
        if mid not in datos['Materias_por_Profesor'][profe_id]:
            datos['Materias_por_Profesor'][profe_id].append(mid)
        if profe_id not in datos['Profesores_por_Materia'][mid]:
            datos['Profesores_por_Materia'][mid].append(profe_id)

materias_sin_profesor = [m for m in datos['id_Materias'] if m not in datos['Profesores_por_Materia']]

profesores_imaginarios = [f"PA_{i}" for i in range(len(materias_sin_profesor))]
datos['Profesores_Imaginarios'] = profesores_imaginarios
print("Se han creado", len(profesores_imaginarios), "profesores imaginarios para las materias sin profesor asignado.")

for i, materia_id in enumerate(materias_sin_profesor):
    profe_imag = profesores_imaginarios[i]
    datos['Profesores_por_Materia'][materia_id].append(profe_imag)
    datos['Materias_por_Profesor'][profe_imag].append(materia_id)
    datos['Tipo_Contrato'][profe_imag] = 4

datos['Profesores_Totales'].extend(datos['Id_Profesores'])
datos['Profesores_Totales'].extend(profesores_imaginarios)

datos['nombres_profesor'].update({
    p: f"ProfFicticio_{i}" for i, p in enumerate(profesores_imaginarios)
})


Se han creado 11 profesores imaginarios para las materias sin profesor asignado.


In [36]:
import pandas as pd

def leer_bloques_desde_excel(ruta_archivo):
    # Leer archivo CSV
    df = pd.read_csv(ruta_archivo)

    print("Columnas en el archivo:", df.columns.tolist())

    bloques = {}

    for _, fila in df.iterrows():
        programa = fila['Programa']
        materia = fila['Materia']
        bloque = fila['Bloque']
        cupo = fila['Cupo']

        if bloque not in bloques:
            bloques[bloque] = {
                'programa_educativo': programa,
                'materias': [],
                'cupo': cupo
            }
        bloques[bloque]['materias'].append(materia)

    return bloques

ruta = r"C:\Users\bdgae\Desktop\Nueva carpeta\exportado.csv"
datos['Bloques'] = leer_bloques_desde_excel(ruta)


Columnas en el archivo: ['Programa', 'Materia', 'Bloque', 'Cupo']


In [37]:
print(type(datos))  # Verifica el tipo de 'datos'
print(datos.keys())  # Muestra las claves del diccionario
print(len(datos))    # Muestra cuántos elementos hay en el diccionario

<class 'dict'>
dict_keys(['Id_Profesores', 'nombres_profesor', 'Tipo_Contrato', 'Detalles_Contratos', 'id_Materias', 'nombres_materia', 'Horas_Semana_Materia', 'Tipo_Clase', 'Programa_Educativo_Materia', 'Aulas', 'Tipo_Aula', 'Id_Edificios', 'Edificios', 'Dia', 'Hora', 'Compatibiliadad_Aula_Materia', 'Materias_por_Profesor', 'Profesores_por_Materia', 'Profesores_Imaginarios', 'Profesores_Totales', 'Bloques'])
21


In [38]:
datos['Bloques']

{'ARQBLO1': {'programa_educativo': 'ARQ',
  'materias': ['Teoría de la Arquitectura',
   'Investigación para la Arquitec',
   'Hist. y Tendencias Urbano Arq.',
   'Dibujo I',
   'Geometría Descriptiva',
   'Taller de Diseño Básico',
   'Matem. para la Arquitectura I',
   'Introducción a la FGU',
   'Inglés I',
   'Arquitectura de la Antigüedad',
   'Métodos y Estrat. Proyectuales',
   'P. Inclusión Soc.en el Espacio',
   'Dibujo II',
   'Taller de Diseño Integral I',
   'Matem. Aplicadas a la Arqui.II',
   'Introducc. a las Instalaciones',
   'Conc. Básicos de Construcción',
   'Inglés II'],
  'cupo': 50},
 'ARQBLO2': {'programa_educativo': 'ARQ',
  'materias': ['Teoría de la Arquitectura',
   'Investigación para la Arquitec',
   'Hist. y Tendencias Urbano Arq.',
   'Dibujo I',
   'Geometría Descriptiva',
   'Taller de Diseño Básico',
   'Matem. para la Arquitectura I',
   'Introducción a la FGU',
   'Inglés I',
   'Arquitectura de la Antigüedad',
   'Métodos y Estrat. Proyectuales',
 

In [39]:
# Imprimir todas las claves (columnas) del diccionario 'datos' y el primer elemento de cada una (si aplica)
print("Columnas (claves) en 'datos':")
for key in datos.keys():
    print(f"{key}: ", end="")
    valor = datos[key]
    # Si es lista y no está vacía, imprime el primer elemento
    if isinstance(valor, list) and len(valor) > 0:
        print(valor[0])
    # Si es dict y no está vacío, imprime el primer par clave-valor
    elif isinstance(valor, dict) and len(valor) > 0:
        primera_clave = next(iter(valor))
        print(f"{primera_clave}: {valor[primera_clave]}")
    # Si es defaultdict y no está vacío, imprime el primer par clave-valor
    elif "defaultdict" in str(type(valor)) and len(valor) > 0:
        primera_clave = next(iter(valor))
        print(f"{primera_clave}: {valor[primera_clave]}")
    else:
        print(valor)

Columnas (claves) en 'datos':
Id_Profesores: 100002999
nombres_profesor: 100002999: GARCIA - ZENTENO EDUARDO
Tipo_Contrato: 100002999: 3
Detalles_Contratos: 1: {'min': 12, 'max': 20}
id_Materias: 1
nombres_materia: 1: Teoría de la Arquitectura
Horas_Semana_Materia: 1: 3
Tipo_Clase: 1: 4
Programa_Educativo_Materia: 1: 5
Aulas: (101, 2)
Tipo_Aula: (101, 2): 3
Id_Edificios: 1
Edificios: 1: EMA1
Dia: 1
Hora: 1
Compatibiliadad_Aula_Materia: ((101, 2), 1): {'compatible': 0, 'tipo_materia': 4, 'tipo_aula': 3}
Materias_por_Profesor: 100004288: [1, 2, 3]
Profesores_por_Materia: 1: [100004288, 100020344]
Profesores_Imaginarios: PA_0
Profesores_Totales: 100002999
Bloques: ARQBLO1: {'programa_educativo': 'ARQ', 'materias': ['Teoría de la Arquitectura', 'Investigación para la Arquitec', 'Hist. y Tendencias Urbano Arq.', 'Dibujo I', 'Geometría Descriptiva', 'Taller de Diseño Básico', 'Matem. para la Arquitectura I', 'Introducción a la FGU', 'Inglés I', 'Arquitectura de la Antigüedad', 'Métodos y Est

## Crear Modelo de Pyomo

---

### Conjuntos

- $P: \text{Profesores}$
    - $P_R: \text{Profesores Reales}$
    - $P_I: \text{Profesores Imaginarios}$
    - $P = P_R \cup\ P_I$
- $M: \text{Materias}$
- $A: \text{Aulas}$
    - $A = A_{salon}\ \cup\ A_{edificio}$
- $BH: \text{Bloques Horarios}$
    - $D: \text{Dias}, D = \{1, 2,\ \dots, 5\}$
    - $H: \text{Horarios}, H = \{1, 2,\ \dots, 10\}$
    - $BH = D \times H = \{(d, h)\ |\ d \in D, h \in H\}$

In [40]:
# Definir los conjuntos
model = pyo.ConcreteModel()

# Conjuntos
model.Profesores = pyo.Set(initialize=datos['Profesores_Totales'])
model.Materias = pyo.Set(initialize=datos['id_Materias'])
model.Aulas = pyo.Set(dimen=2, initialize=[(aula_id, edificio_id) for aula_id, edificio_id in datos['Aulas']])
model.BloquesHorarios = pyo.Set(dimen=2, initialize=[(dia, hora) for dia in datos['Dia'] for hora in datos['Hora']])
model.Dias = pyo.Set(initialize=datos['Dia'])
model.Hora = pyo.Set(initialize=datos['Hora'])

### Parametros

TODO: AGREGAR FUNCION PARA DETECTAR PROF. IMAGINARIOS
- $\text{HorasSemanaMateria}(m) = n$, donde $n$ es el numero de horas que la materia $m$ tiene por semana
- $\text{TipoClase}(m) = t$, donde $t \in \{1,2,3,4\}$ es el tipo de clase de la materia $m$
- $\text{TipoAula}(a) = t$, donde $t \in \{1,2,3,4\}$ es el tipo de aula de la aula $a$
- $ \text{Compatibles}(a, m) \begin{cases} 1 & \text{si el aula}\ a\ \text{es compatible con la materia}\ m\\ 0 & \text{si no} \end{cases}$

- $ \text{Relacion}(p, m) \begin{cases} 1 & \text{si el profesor}\ p\ \text{puede dar la materia}\ m\\ 0 & \text{si no} \end{cases}$

In [41]:
# Parámetros

model.Horas_Semana_Materia = pyo.Param(model.Materias, initialize=datos['Horas_Semana_Materia'])
model.Tipo_Clase = pyo.Param(model.Materias, initialize=datos['Tipo_Clase'])
model.Tipo_Aula = pyo.Param(model.Aulas, initialize=datos['Tipo_Aula'])
model.Compatibilidad_Aula_Materia = pyo.Param(model.Aulas, model.Materias, initialize={
    (aula, materia): datos['Compatibiliadad_Aula_Materia'].get((aula, materia), {}).get('compatible', 0)
    for aula in model.Aulas for materia in model.Materias
})  # Compatibilidad entre aulas y materias (0 o 1)
# Relación entre profesores y las materias que pueden enseñar
model.Relacion_Profesor_Materia = pyo.Param(model.Profesores, model.Materias, initialize={
    (profe_id, materia_id): 1 if materia_id in datos['Materias_por_Profesor'][profe_id] else 0
    for profe_id in model.Profesores for materia_id in model.Materias
})





### Variables

- $
x^{i, j}_{k, t} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ tiene asignada la materia } j
    \text{ en el aula } k
    \text{ en el horario } t \\
    0 & \text{si no}
\end{cases}
$

- $
y_{i, j} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ puede enseñar la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
\hat{y}_{i, j} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ tiene asignada la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
z_{k, j} =
\begin{cases}
    1 & \text{si el aula } k
    \text{ esta asignada a la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
\hat{z}_{j, d} =
\begin{cases}
    1 & \text{si la materia } j
    \text{ se imparte el dia } d \\
    0 & \text{si no}
\end{cases}
$

- $
s_{j, t, d} =
\begin{cases}
    1 & \text{si el curso } j
    \text{ comienza a la hora } t
    \text{ en el dia } d \\
    0 & \text{si no}
\end{cases}
$

In [42]:
# Variables

X_index = [
    (p, m, a, bh)
    for p in datos['Profesores_Totales']  # Cambiado a Profesores_Totales (incluye reales e imaginarios)
    for m in datos['id_Materias']
    if m in datos['Materias_por_Profesor'].get(p, [])  # Usar get para evitar KeyError
    for a in datos['Aulas']
    if datos['Compatibiliadad_Aula_Materia'].get((a, m), {}).get('compatible', 0) == 1  # Aulas compatibles
    for bh in [(d, h) for d in datos['Dia'] for h in datos['Hora']]
]

PROFESOR = 0
MATERIA = 1
AULA = 2
EDIFICIO = 3
DIA = 4
HORA = 5

# X_Index 
#(100004288, 1, (102, 5), (1, 2))
# Model.X tiene la siguiente estructura para sus indices, pyomo aplana las tuplas :
# (100004288, 1, 102, 5, 1, 1) 

model.X = pyo.Var(X_index, domain=pyo.Binary)

# Variables generales: profesor puede enseñar materia (relación) - incluye imaginarios
Y_index = [(p, m) for p in datos['Profesores_Totales'] for m in datos['Materias_por_Profesor'].get(p, [])]
model.Y = pyo.Var(Y_index, domain=pyo.Binary)

# Variables generales: profesor tiene asignada la materia
Y_hat_index = [(p, m) for p in datos['Profesores_Totales'] for m in datos['id_Materias']]
model.Y_hat = pyo.Var(Y_hat_index, domain=pyo.Binary)

# Variables generales: aula asignada a materia (sin cambios)
Z_index = [(aula, edificio, materia) 
           for ((aula, edificio), materia), val in datos['Compatibiliadad_Aula_Materia'].items() 
           if val['compatible'] == 1]
model.Z = pyo.Var(Z_index, domain=pyo.Binary)

# Variables generales: materia impartida en dia d
Z_hat_index = [(m, d) for m in datos['id_Materias'] for d in datos['Dia']]
model.Z_hat = pyo.Var(Z_hat_index, domain=pyo.Binary)

# # Variable Materia X es dada en hora H el dia D
S_Index = [(m, h, d) for m in datos['id_Materias'] for h in datos['Hora'] for d in datos['Dia']]
model.S = pyo.Var(S_Index, domain=pyo.Binary)





print(f"Variables reducidas de {len(datos['Profesores_Totales'])*len(datos['id_Materias'])*len(datos['Aulas'])*len(datos['Dia'])*len(datos['Hora'])} a {len(X_index)}")

Variables reducidas de 64935000 a 1529050


### Restricciones

- El curso $j$ debe tener asignado un profesor, y el profesor debe poder impartir la materia:
$$
\sum_{i \in P} \hat{y}_{i, j} \cdot \text{Relacion}(i, j) = 1,\ \forall\ j \in M
$$

In [43]:
#Funciona Todas las materias tiene profesor
if hasattr(model, 'Restriccion_Materia_Tiene_Profe'):
    model.del_component(model.Restriccion_Materia_Tiene_Profe)

def materia_tiene_profe_rule(model, m):
    return sum(
        model.Y_hat[p, m] * model.Relacion_Profesor_Materia[p, m]
        for p in model.Profesores
    ) == 1

model.Restriccion_Materia_Tiene_Profe = pyo.Constraint(model.Materias, rule=materia_tiene_profe_rule)

- El profesor $i$ debe impartir todas las horas del curso $j$, si el es seleccionada para impartirlo:

$$
\sum_{k \in A} \sum_{t \in BH} x^{i, j}_{k, t} = \text{HorasSemanaMateria}(j) \cdot \hat{y}_{i, j} \quad \forall\ j \in M, i \in P
$$

In [44]:
#Funciona Se da el numero total de horas semanales requeridas por cada materia
if hasattr(model, 'Restriccion_Horas_Semanales'):
    model.del_component(model.Restriccion_Horas_Semanales)

# Precomputar la suma de X[p, m] por cada profesor y materia
suma_X = {}

for idx in model.X:
    profesor = idx[PROFESOR]    # El índice 0 corresponde a profesor
    materia = idx[MATERIA]      # El índice 1 corresponde a materia
    aula = idx[AULA]            # El índice 2 corresponde a aula
    dia = idx[DIA]              # El índice 3 corresponde a día
    hora = idx[HORA]            # El índice 4 corresponde a hora
    bh = idx[EDIFICIO]          # El índice 5 corresponde a edificio (bh)

    if (profesor, materia) not in suma_X:
        suma_X[(profesor, materia)] = 0
    
    suma_X[(profesor, materia)] += model.X[idx]



def horas_semana_materia_rule(model, p, m):
    # Obtener la suma precomputada de X[p, m]
    suma = suma_X.get((p, m), 0)  # Si no existe, retornar 0
    return suma == datos['Horas_Semana_Materia'][m] * model.Y_hat[p, m]

model.Restriccion_Horas_Semanales = pyo.Constraint(
    datos['Profesores_Totales'], 
    datos['id_Materias'], 
    rule=horas_semana_materia_rule
)



- No Solapamiento Aula (No asignar un aula a más de una materia en un bloque horario):
$$
\sum_{i \in P} \sum_{j \in M} x^{i, j}_{k, t} \le 1 \quad \forall\ k \in A,\ t \in BH
$$

In [45]:
#Funciona No solapamiento de aulas
if hasattr(model, 'Restriccion_Aula_Unica'):
    model.del_component(model.Restriccion_Aula_Unica)

# Precomputar las combinaciones de aula, edificio, día y hora
solapamientos = {}

for idx in model.X:
    aula = idx[AULA]
    edificio = idx[EDIFICIO]
    dia = idx[DIA]
    hora = idx[HORA]
    
    if (aula, edificio, dia, hora) not in solapamientos:
        solapamientos[(aula, edificio, dia, hora)] = []

    solapamientos[(aula, edificio, dia, hora)].append(model.X[idx])

# Restricción para el no solapamiento de aulas
def no_solapamiento_aula_rule(model, aula_id, edificio_id, dia, hora):
    if (aula_id, edificio_id, dia, hora) not in solapamientos:
        return pyo.Constraint.Skip  # Si no existe ninguna asignación para esa combinación

    exprs = solapamientos[(aula_id, edificio_id, dia, hora)]
    return sum(exprs) <= 1

# Aplicamos la restricción optimizada
model.Restriccion_Aula_Unica = pyo.Constraint(
    model.Aulas, model.BloquesHorarios,
    rule=no_solapamiento_aula_rule
)




Escribirle las matematicas

In [46]:
# Precalcular los índices por profesor
indices_por_profesor = {p: [] for p in datos['Profesores_Totales']}

for idx in model.X:
    profesor = idx[PROFESOR]
    if profesor in indices_por_profesor:
        indices_por_profesor[profesor].append(idx)

# Eliminar restricciones si ya existen
if hasattr(model, 'Restriccion_Max_Horas_Profesor'):
    model.del_component(model.Restriccion_Max_Horas_Profesor)


tipo_contrato_por_profesor = datos['Tipo_Contrato']
detalles_contratos = datos['Detalles_Contratos']

# Restricción máxima
def max_horas_profesor_rule(model, p):
    if (
        p not in tipo_contrato_por_profesor
        or p not in indices_por_profesor
        or not indices_por_profesor[p]  # Lista vacía
    ):
        return pyo.Constraint.Skip

    contrato_id = tipo_contrato_por_profesor[p]
    max_horas = detalles_contratos[contrato_id]['max']

    return sum(model.X[idx] for idx in indices_por_profesor[p]) <= max_horas


model.Restriccion_Max_Horas_Profesor = pyo.Constraint(model.Profesores, rule=max_horas_profesor_rule)



In [47]:
#Restringue que cada materia sea asignada a un aula unica

if hasattr(model, 'Restriccion_Ubicacion_Unica_Materia'):
    model.del_component(model.Restriccion_Ubicacion_Unica_Materia)

def ubicacion_unica_por_materia_rule(model, m):
    return sum(model.Z[aula, edificio, m] for (aula, edificio, mm) in model.Z.index_set() if mm == m) == 1

model.Restriccion_Ubicacion_Unica_Materia = pyo.Constraint(
    model.Materias, rule=ubicacion_unica_por_materia_rule
)


TODO: ESCRIBIR RESTRICCION VARIABLE S

- No Solapamiento Profesor (No asignar un profesor a más de una materia en un bloque horario):
$$
\sum_{j \in M} \sum_{k \in A} x^{i, j}_{k, t} \le 1 \quad \forall\ i \in P,\ t \in BH

In [48]:
#Funciona Ningún profesor puede impartir más de una materia en el mismo bloque horario (Falta checar con mas datos)
if hasattr(model, 'Restriccion_Profesor_Unico'):
    model.del_component(model.Restriccion_Profesor_Unico)


# Precomputar los solapamientos de profesores por día y hora
solapamientos_profesor = {}

for idx in model.X:
    profesor = idx[PROFESOR]
    dia = idx[DIA]
    hora = idx[HORA]
    
    if (profesor, dia, hora) not in solapamientos_profesor:
        solapamientos_profesor[(profesor, dia, hora)] = []

    solapamientos_profesor[(profesor, dia, hora)].append(model.X[idx])

def no_solapamiento_profesor_rule(model, p, d, h):
    if (p, d, h) not in solapamientos_profesor:
        return pyo.Constraint.Skip
    exprs = solapamientos_profesor[(p, d, h)]
    return sum(exprs) <= 1

model.Restriccion_Profesor_Unico = pyo.Constraint(
    model.Profesores, model.Dias, datos['Hora'],
    rule=no_solapamiento_profesor_rule
)


- Profesor Puede Enseñar (Asignar un profesor a una materia solo si el profesor puede enseñar esa materia):
$$
x^{i, j}_{k, t} \le \text{Relacion}(i,j) \quad \forall\ i \in P,\ j \in M,\ k \in A,\ t \in BH
$$

In [49]:
#Funciona Un profesor solo puede impartir una materia 
if hasattr(model, 'Restriccion_Profesor_Compatible'):
    model.del_component(model.Restriccion_Profesor_Compatible)

# Precomputar la relación de profesor y materia
profesor_materia_relacion = {}

for idx in model.X:
    profesor = idx[PROFESOR]
    materia = idx[MATERIA]
    
    if (profesor, materia) not in profesor_materia_relacion:
        profesor_materia_relacion[(profesor, materia)] = model.Relacion_Profesor_Materia[profesor, materia]

def profesor_puede_ensenar_rule(model, p, m, id_aula, id_edificio, dia, hora):
    return model.X[p, m, id_aula, id_edificio, dia, hora] <= profesor_materia_relacion.get((p, m), 0)

model.Restriccion_Profesor_Compatible = pyo.Constraint(
    model.X.index_set(),
    rule=profesor_puede_ensenar_rule
)


- Aula Compatible Materia (Asignar un aula a una materia solo si el tipo de aula es compatible con el tipo de materia):

$$
x^{i, j}_{k, t} \le \text{Compatibles}(j,k) \quad \forall\ i \in P,\ j \in M,\ k \in A,\ t \in BH
$$

In [50]:
#Funciona Restricción de compatibilidad entre aula y materia (checar con mas datos)
if hasattr(model, 'Restriccion_Aula_Compatible'):
    model.del_component(model.Restriccion_Aula_Compatible)

# Precomputar la compatibilidad entre aula y materia
aula_materia_compatibilidad = {}

for idx in model.X:
    aula = (idx[AULA], idx[EDIFICIO])
    materia = idx[MATERIA]
    
    if (aula, materia) not in aula_materia_compatibilidad:
        aula_materia_compatibilidad[(aula, materia)] = model.Compatibilidad_Aula_Materia[aula, materia]

def aula_compatible_materia_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    return model.X[p, m, aula, (dia, hora)] <= aula_materia_compatibilidad.get((aula, m), 0)

model.Restriccion_Aula_Compatible = pyo.Constraint(
    model.X.index_set(),
    rule=aula_compatible_materia_rule
)

- Si hay una asignacion para una clase a una hora de un maestro, que la materia este

$$
x^{i,j}_{k,t} \le z_{j,k} \quad i \in P, j \in M, k \in A, t \in BH
$$

In [51]:
#Funciona Relación entre X y Z (aula asignada a materia) Se hizo bidirectional para que se pueda usar en ambos sentidos
if hasattr(model, 'Relacion_X_Z'):
    model.del_component(model.Relacion_X_Z)

# Precomputación de compatibilidad entre X y Z para todos los posibles (aula, materia)
aula_materia_z = {}

for idx in model.X:
    aula = (idx[AULA], idx[EDIFICIO])
    materia = idx[MATERIA]
    
    if (aula, materia) not in aula_materia_z:
        aula_materia_z[(aula, materia)] = model.Z[aula, materia]

def relacion_X_Z_rule_1(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    
    # Restricción: Si X(p, m, aula, bloque_horario) = 1, entonces Z(aula, m) = 1
    return model.X[p, m, aula, bloque_horario] <= aula_materia_z.get((aula, m), 0)

def relacion_X_Z_rule_2(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    
    # Restricción: Si Z(aula, m) = 0, entonces X(p, m, aula, bloque_horario) = 0
    return model.Z[aula, m] >= model.X[p, m, aula, bloque_horario]

model.Relacion_X_Z_1 = pyo.Constraint(X_index, rule=relacion_X_Z_rule_1)
model.Relacion_X_Z_2 = pyo.Constraint(X_index, rule=relacion_X_Z_rule_2)



- Las sesiones del curso $j$ deben tener una duracion de al menos $\psi_{min}(j)$ hrs y no mas de
$\psi_{max}(j)$ hrs:
$$
\psi_{min}(j) \cdot z_{j, d} \le
\sum_{i \in P} \sum_{k \in A} \sum_{h \in H} x^{i, j}_{k, (d, h)} \le
\psi_{max}(j) \cdot z_{j, d} \\[0.5em]
j \in M,\ d \in D
$$

- Solo haya una sesion por dia para una materia $j$ en un dia $d$
$$
\sum_{t \in T} s_{j, t, d} \le \hat{z}_{j, d} \quad j \in M,\ d \in D
$$

- Restriccion si una materia inicia en una hora $t$, que la variable $s$ marque su inicio
$$
s_{j, t, d} \ge \sum_{k \in A} x^{i, j}_{k, t} - \sum_{k \in A} x^{i, j}_{k, t'} \\[0.5em]
t' = (d, t -1),\ i \in P,\ j \in M,\ t \in H\setminus 1,\ d \in D
$$

In [52]:
# --- Restricción: Las clases de una materia en un día deben ser consecutivas ---

# Eliminar componente anterior si ya existe
if hasattr(model, 'Restriccion_Consecutividad_Materia_Dia'):
    model.del_component(model.Restriccion_Consecutividad_Materia_Dia)

# Precalcular clases posibles por (materia, día, hora)
clases_por_materia_dia_hora = {}

for (p, m, aula, edificio, d, h) in model.X.index_set():
    key = (m, d, h)
    if key not in clases_por_materia_dia_hora:
        clases_por_materia_dia_hora[key] = []
    clases_por_materia_dia_hora[key].append(model.X[p, m, aula, edificio, d, h])

# Definir el conjunto de índices válidos (no incluye la última hora del día)
consecutividad_index = [
    (m, d, h)
    for m in datos['id_Materias']
    for d in datos['Dia']
    for h in datos['Hora'][:-1]
]

# Regla: si hay clase en h+1, debe haber clase en h
def consecutividad_materia_dia_rule(model, m, d, h):
    clases_h = clases_por_materia_dia_hora.get((m, d, h), [])
    clases_h1 = clases_por_materia_dia_hora.get((m, d, h + 1), [])

    # Si no hay clases posibles en h+1, no aplicar la restricción
    if not clases_h1:
        return pyo.Constraint.Skip

    return sum(clases_h1) <= sum(clases_h)

# Crear restricción en el modelo
model.Restriccion_Consecutividad_Materia_Dia = pyo.Constraint(
    consecutividad_index,
    rule=consecutividad_materia_dia_rule
)


In [53]:
# # --- Restricción: Todas las clases de una materia en un día deben ser consecutivas ---

# # Para cada materia m, día d y cada hora h (excepto la última), si hay clase en h+1, debe haber clase en h.
# # Se define una restricción para cada (m, d, h) con h en todas menos la última.

# if hasattr(model, 'Restriccion_Consecutividad_Materia_Dia'):
#     model.del_component(model.Restriccion_Consecutividad_Materia_Dia)

# def consecutividad_materia_dia_rule(model, m, d, h):
#     # Sumamos sobre todos los profesores y aulas posibles para esa materia, día y hora
#     suma_h = sum(
#         model.X[p, m, aula, edificio, d, h]
#         for (p, mm, aula, edificio, dd, hh) in model.X.index_set()
#         if mm == m and dd == d and hh == h
#     )
#     suma_h1 = sum(
#         model.X[p, m, aula, edificio, d, h+1]
#         for (p, mm, aula, edificio, dd, hh) in model.X.index_set()
#         if mm == m and dd == d and hh == h+1
#     )
#     # Si hay clase en h+1, debe haber clase en h
#     return suma_h1 <= suma_h + (1 - suma_h1)

# # Creamos el conjunto de índices válidos (m, d, h) con h en todas menos la última
# consecutividad_index = [
#     (m, d, h)
#     for m in datos['id_Materias']
#     for d in datos['Dia']
#     for h in datos['Hora'][:-1]
# ]

# model.Restriccion_Consecutividad_Materia_Dia = pyo.Constraint(
#     consecutividad_index, rule=consecutividad_materia_dia_rule
# )


In [54]:
if hasattr(model, 'objetivo_dummy'):
    model.del_component(model.objetivo_dummy)

model.objetivo_dummy = pyo.Objective(expr=1, sense=pyo.minimize)

In [55]:
# Obtener todas las restricciones activas del modelo y almacenarlas en una lista
restricciones = []

print("Restricciones definidas en el modelo:")
for nombre, componente in model.component_map(pyo.Constraint, active=True).items():
    restricciones.append(nombre)
    print(f"- {nombre}")

# Ahora tienes una lista con los nombres de las restricciones
# Puedes usarla después si necesitas procesarlas de otra forma


Restricciones definidas en el modelo:
- Restriccion_Materia_Tiene_Profe
- Restriccion_Horas_Semanales
- Restriccion_Aula_Unica
- Restriccion_Max_Horas_Profesor
- Restriccion_Ubicacion_Unica_Materia
- Restriccion_Profesor_Unico
- Restriccion_Profesor_Compatible
- Restriccion_Aula_Compatible
- Relacion_X_Z_1
- Relacion_X_Z_2
- Restriccion_Consecutividad_Materia_Dia


In [56]:
def resolver_modelo(model, mipgap=0.02, threads=4, emphasis=1):
    from pyomo.opt import SolverFactory, TerminationCondition

    solver = SolverFactory('cplex')

    if not solver.available():
        raise RuntimeError("CPLEX no está disponible. Verifica la instalación o el PATH.")

    resultado = solver.solve(model, tee=True)

    # Verificar si la solución fue óptima o factible
    condicion = resultado.solver.termination_condition
    if condicion in [TerminationCondition.optimal, TerminationCondition.feasible]:
        print(f"Solución encontrada: {condicion}")
    else:
        print(f"Problema durante la resolución: {condicion}")
        print("Revisa el modelo o ajusta parámetros del solver.")

    return resultado


In [57]:
import pandas as pd
from collections import defaultdict

def mostrar_bloque(dia, hora_inicio, hora_fin):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    dia_str = dias[dia - 1]
    hora_inicio_real = 7 + hora_inicio
    hora_fin_real = 7 + hora_fin
    return f"{dia_str} {hora_inicio_real:02d}:00 - {hora_fin_real:02d}:00"

def mostrar_asignacion(model):
    sesiones = defaultdict(list)

    # Agrupar sesiones por clave común
    for idx in model.X:
        if pyo.value(model.X[idx]) > 0.5:
            p, m, aula, edificio_id, dia, hora = idx
            clave = (p, m, aula, edificio_id, dia)
            sesiones[clave].append(hora)

    asignaciones = []

    print("\nAsignaciones de profesores, materias, aulas y horarios:")
    for (p, m, aula, edificio_id, dia), horas in sesiones.items():
        # Agrupar bloques consecutivos
        horas = sorted(horas)
        bloques = []
        inicio = fin = horas[0]
        for h in horas[1:]:
            if h == fin + 1:
                fin = h
            else:
                bloques.append((inicio, fin + 1))
                inicio = fin = h
        bloques.append((inicio, fin + 1))  # agregar el último bloque

        # Mostrar
        profesor_nombre = datos['nombres_profesor'].get(p, p)
        materia_nombre = datos['nombres_materia'].get(m, m)
        edificio_nombre = datos['Edificios'].get(edificio_id, f"Edificio {edificio_id}")

        for hora_inicio, hora_fin in bloques:
            horario_str = mostrar_bloque(dia, hora_inicio, hora_fin)
            print(f"Profesor: {profesor_nombre} | Materia: {materia_nombre} | "
                  f"Aula: {aula} (Edificio {edificio_nombre}) | Horario: {horario_str}")

            asignaciones.append({
                'id_materia': m,
                'Programa_Educativo_Materia': datos['Programa_Educativo_Materia'].get(m),
                'Materia': materia_nombre,
                'Profesor': profesor_nombre,
                'Edificio': edificio_nombre,
                'Aula': aula,
                'Horario': horario_str
                })


    # Exportar a Excel
    df = pd.DataFrame(asignaciones, columns=[
        'id_materia',
        'Programa_Educativo_Materia',
        'Materia',
        'Profesor',
        'Edificio',
        'Aula',
        'Horario'
    ])
    df.to_excel('asignaciones.xlsx', index=False)
    print("\nArchivo 'asignaciones.xlsx' creado con éxito.")

    return asignaciones


In [58]:
# Resolver el modelo
resultados = resolver_modelo(model)

# Verificar si encontró solución
if (resultados.solver.status == pyo.SolverStatus.ok and
    resultados.solver.termination_condition == TerminationCondition.optimal):

    print("Solución óptima encontrada.\n")
    asignaciones_lista = mostrar_asignacion(model)

    import pandas as pd
    df = pd.DataFrame(asignaciones_lista)
    profesores_usados = {a['Profesor'] for a in asignaciones_lista}
    total_usados = len(profesores_usados)
    total_profesores = datos['Profesores_Totales']

    print(f"\nProfesores asignados: {total_usados} de {len(total_profesores)}")

    
    solapes_profesor = df[df.duplicated(['Profesor', 'Horario'], keep=False)]
    solapes_aula = df[df.duplicated(['Aula','Edificio','Horario'], keep=False)]

    if solapes_profesor.empty:
        print("✅ No hay solapamientos de profesor.")
    else:
        print("⚠️ Hay solapamientos de profesor:")

    if solapes_aula.empty:
        print("✅ No hay solapamientos de aula.")
    else:
        print("⚠️ Hay solapamientos de aula:")
        print(solapes_aula)

else:
    print("No se encontró solución óptima.")
    print("Estado:", resultados.solver.status)
    print("Condición de terminación:", resultados.solver.termination_condition)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\bdgae\AppData\Local\Temp\tmp1l5hv58u.cplex.log' open.
CPLEX> Problem 'C:\Users\bdgae\AppData\Local\Temp\tmp2rk98coi.pyomo.lp' read.
Read time = 18.31 sec. (585.18 ticks)
CPLEX> Problem name         : C:\Users\bdgae\AppData\Local\Temp\tmp2rk98coi.pyomo.lp
Objective sense      : Minimize
Variables            : 1546419  [Fix: 1,  Binary: 1546418]
Objective nonzeros   :       1
Linear constraints   : 6139570  [Less: 6128270,  Equal: 11300]
  Nonzeros           : 18060637
  RHS nonzeros       : 3065870

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonze

In [89]:
import re

def revisar_restricciones(df):
    errores = []

    # Asegurar que 'Día' exista
    if 'Día' not in df.columns:
        df['Día'] = df['Horario'].apply(lambda h: h.split()[0])

    # Extraer hora de inicio
    def extraer_hora_inicio(horario):
        match = re.search(r'(\d{2}):\d{2}', horario)
        return int(match.group(1)) if match else None

    df['Hora_inicio'] = df['Horario'].apply(extraer_hora_inicio)

    # Clave principal para agrupación: (id_materia, Programa_Educativo_Materia)
    clave_materia = ['id_materia', 'Programa_Educativo_Materia']

    # 1. Materias sin profesor
    sin_profe = df[df['Profesor'].isnull()][clave_materia].drop_duplicates()
    if not sin_profe.empty:
        errores.append(f"Materias sin profesor asignado:\n{sin_profe.to_dict(orient='records')}")

    # 2. Materias con menos de 1 horario
    horas_por_materia = df.groupby(clave_materia)['Horario'].nunique()
    pocas_horas = horas_por_materia[horas_por_materia < 1]
    if not pocas_horas.empty:
        errores.append(f"Materias con menos horas semanales:\n{pocas_horas.index.tolist()}")

    # 3. Materias en múltiples aulas
    aulas_por_materia = df.groupby(clave_materia)['Aula'].nunique()
    multiples_aulas = aulas_por_materia[aulas_por_materia > 1]
    if not multiples_aulas.empty:
        errores.append(f"Materias en múltiples aulas:\n{multiples_aulas.index.tolist()}")

    # 4. Profesores con más de 10 horarios
    max_horas = 10
    horas_por_profesor = df.groupby('Profesor')['Horario'].nunique()
    excedidos = horas_por_profesor[horas_por_profesor > max_horas]
    if not excedidos.empty:
        errores.append(f"Profesores con más de {max_horas} horarios:\n{excedidos.index.tolist()}")

    # 5. Materias en múltiples edificios
    edificios_por_materia = df.groupby(clave_materia)['Edificio'].nunique()
    multiples_edificios = edificios_por_materia[edificios_por_materia > 1]
    if not multiples_edificios.empty:
        errores.append(f"Materias en múltiples edificios:\n{multiples_edificios.index.tolist()}")

    # 6. Materias con más de un profesor
    profes_por_materia = df.groupby(clave_materia)['Profesor'].nunique()
    varios_profes = profes_por_materia[profes_por_materia > 1]
    if not varios_profes.empty:
        errores.append(f"Materias con más de un profesor asignado:\n{varios_profes.index.tolist()}")

    # 7. Profesores con clases simultáneas
    profe_horario = df.groupby(['Profesor', 'Horario']).size()
    conflictos_profe = profe_horario[profe_horario > 1]
    if not conflictos_profe.empty:
        errores.append(f"Profesores con clases simultáneas:\n{conflictos_profe.index.tolist()}")

    # 8. Aulas con clases simultáneas
    aula_horario = df.groupby(['Aula','Edificio','Horario']).size()
    conflictos_aula = aula_horario[aula_horario > 1]
    if not conflictos_aula.empty:
        errores.append(f"Aulas con clases simultáneas:\n{conflictos_aula.index.tolist()}")

    # 11. Clases no consecutivas por materia y día
    for (m_id, programa, dia), grupo in df.groupby(clave_materia + ['Día']):
        horas = sorted(grupo['Hora_inicio'].dropna().unique())
        if len(horas) > 1:
            consecutivas = all(horas[i+1] == horas[i] + 1 for i in range(len(horas)-1))
            if not consecutivas:
                errores.append(f"Clases no consecutivas para materia {m_id} (Programa {programa}) el día {dia}: {horas}")

    # Resultados
    if errores:
        print("❌ Restricciones violadas:")
        for e in errores:
            print(f"- {e}")
    else:
        print("✅ Todas las restricciones se cumplen.")

# Ejecutar verificación
revisar_restricciones(df)


✅ Todas las restricciones se cumplen.
